# Using **EigenvalueSolver.jl**

This is a demo file illustrating how to use the functions in **EigenvalueSolver.jl**.
More details can be found in [1, Section 4].

[1] M. R. Bender and S. Telen. "Yet another eigenvalue algorithm for solving polynomial systems." *arXiv preprint arXiv:2105.08472* (2021). https://arxiv.org/abs/2105.08472

In [ ]:
include("../src/EigenvalueSolver.jl")
using DynamicPolynomials

# To install "SmithNormalForm.jl" use the following command
# Pkg.add(url = "https://github.com/wildart/SmithNormalForm.jl.git")

## Square systems

*The number of equations agrees with the number of variables*

Construct a polynomial system ${\cal F} = (f_1, f_2)$ given by two plane curves of degree 20. 

In [ ]:
@polyvar x[1:2]
ds = [20;20]
f = EigenvalueSolver.getRandomSystem_dense(x,ds)

Compute all intersection points of these two curves.

In [ ]:
@time sol = EigenvalueSolver.solve_CI_dense(f,x; DBD = false, verbose = true);

Compute the backward error of all computed solutions

In [ ]:
BWEs = EigenvalueSolver.get_residual(f,sol,x)
BWE = maximum(BWEs)

Consider a dense, 3-dimensional system ${\mathcal F} = \{f_1,f_2,f_3\}$ given by equations of degree 4, 8 and 12. 

In [ ]:
@polyvar x[1:3]
ds = [4;8;12] # degrees of the equations
f = EigenvalueSolver.getRandomSystem_dense(x,ds)
@time sol = EigenvalueSolver.solve_CI_dense(f,x, verbose = true)
BWEs = EigenvalueSolver.get_residual(f,sol,x) # compute the relative backward error of all solutions
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")

The package provides a specialized function to generate and solve *unmixed systems*.

In [ ]:
@polyvar x[1:2];
A = [0 0; 1 0; 1 1; 0 1; 2 2] # monomials whose convex hull is the polytope P
d = [5;12] # equation i has support inside d[i]*P
f = EigenvalueSolver.getRandomSystem_unmixed(x,A,d)
@time sol, A₀, E, D = EigenvalueSolver.solve_CI_unmixed(f,x,A,d; verbose = true)
BWEs = EigenvalueSolver.get_residual(f,sol,x) # compute the relative backward error of all solutions
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")

The function **solve_CI_unmixed** returns an admissible tuple, which can be re-used by plugging it into the function **solve_EV**.

In [ ]:
sol = EigenvalueSolver.solve_EV(f, x, A₀, E, D; verbose = true)
BWEs = EigenvalueSolver.get_residual(f,sol,x) # compute the relative backward error of all solutions
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")

The following lines generate and solve a random square, *multi-graded dense system*.

In [ ]:
@polyvar x[1:4]
vargroups = [[x[1:2]];[x[3:4]]] # groups of variables
ds = [1 6;2 1;3 2;4 1] # multidegrees of the 4 equations in the 2 variable groups
f = EigenvalueSolver.getRandomSystem_multi_dense(vargroups,ds)
@time sol = EigenvalueSolver.solve_CI_multi_dense(f,vargroups,ds;verbose=true)
BWEs = EigenvalueSolver.get_residual(f,sol,x) # compute the relative backward error of all solutions
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")

The package also provides a specialized function to generate and solve *multi-unmixed systems*.

In [ ]:
@polyvar x[1:4]
A1 = [0 0; 1 0; 1 1; 0 1; 2 2]
A2 = 2*[0 0; 1 0; 0 1]
sups = [[A1]; [A2]]
vargroups = [[x[1:2]];[x[3:4]]]
ds = ones(Int,4,2)
f = EigenvalueSolver.getRandomSystem_multi_unmixed(vargroups,sups,ds;complex = false)
@time sol, A₀, E, D = EigenvalueSolver.solve_CI_multi_unmixed(f,vargroups,sups,ds; verbose=true)
BWEs = EigenvalueSolver.get_residual(f,sol,x) # compute the relative backward error of all solutions
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")

The following is a classical example of a square, *mixed system*, coming from molecular biology [2, Section 3.3].

[2] I. Z. Emiris and B. Mourrain. "Computer algebra methods for studying and computing molecular conformations". *Algorithmica*, 25(2):372–402, 1999.

In [ ]:
@polyvar t[1:3]
β = [-13 -1 -1 24 -1; -13 -1 -1 24 -1; -13 -1 -1 24 -1]
mons1 = [1 t[2]^2 t[3]^2 t[2]*t[3] t[2]^2*t[3]^2]
mons2 = [1 t[3]^2 t[1]^2 t[3]*t[1] t[3]^2*t[1]^2]
mons3 = [1 t[1]^2 t[2]^2 t[1]*t[2] t[1]^2*t[2]^2]
f = [β[1,:]'*mons1';β[2,:]'*mons2';β[3,:]'*mons3'][:]
@time sol, A₀, E, D = EigenvalueSolver.solve_CI_mixed(f,t;verbose=true)
BWEs = EigenvalueSolver.get_residual(f,sol,t)
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")

It is possible to re-use the previous computation to avoid computing the admissible tuple again.

In [ ]:
@time EigenvalueSolver.solve_EV(f,t,A₀,E,D;check_criterion = false,verbose=true)
BWEs = EigenvalueSolver.get_residual(f,sol,t)
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")

## Overdetermined systems  [1, Section 4.2]
*The number of equations is larger than the number of variables*

The following lines generate and solve an overdetermined unmixed system in 15 variables with 100 solutions.

The function **solve_OD_unmixed** implements the incremental construction [1, Algorithm 3]

In [ ]:
using LinearAlgebra
@polyvar x[1:15]
A = Matrix(I,15,15)
A = [A[1:14,:]; sum(A[[14;15],:], dims = 1);zeros(1,15)]
A = Int.(A)
A[14,:] = A[14,:]+A[13,:] 
n = 15
F = EigenvalueSolver.getRandomSystem_unmixed(x, A, [2]; complex = true)[1]
mons = monomials(F)
δ = 100
pts = [randn(ComplexF64,n) for i = 1:δ]
println("the system has $δ solutions")
f, κ = EigenvalueSolver.getVanishingPolynomials(pts, mons, x;augm_prec = false)
α = ones(Int,length(f))*2
R_offline = @timed EigenvalueSolver.solve_OD_unmixed(f, x, A, α;complex = true, verbose=true)
BWEs = EigenvalueSolver.get_residual(f,R_offline.value,x)
BWE = maximum(BWEs)
println("Maximal backward error $(BWE)")